# 03Tools - Pipelines Example 2

**Conditionally update endpoint**

>**Note:** Formerly named `03c - BQML + Vertex AI > Pipelines - automated pipelines for updating models`.  This [link](https://github.com/statmike/vertex-ai-mlops/blob/fd442b458c710a0a7afdc41bae690d2a3282e93c/03c%20-%20BQML%20%2B%20Vertex%20AI%20%3E%20Pipelines%20-%20automated%20pipelines%20for%20updating%20models.ipynb) goes to the previous version featured in the video.  The version below has been fully update to match the parameterization of revised notebooks in this `03` series while preserving the orignal operational example - create a challenger model that is used to conditionally update an endpoint.

As time goes on change occurs:
- inputs to our models may shift in distribution compared to when the model was trained - called training-serving skew
- inputs to out models may shift over time - called prediction drift
- new inputs/features may become available
- a better model may be created

In the `03Tools - Predictions` notebook we deployed the model built with BQML in the `03a` notebook to a Vertex AI Endpoint for online prediction.  In this notebook we will build a challenger model with the same training data, also using BQML but with a different model type - a deep neural network similar what we build in the `05` series of notebooks.  We will construct a Vertex AI Pipeline to orchestrate the process of building the new model, comparing to the deployed mode, and conditionally replacing the deployed model with the new one.  

This process could be triggered based on time elapsed, amount of new data, detected training-serving skew or even prediction drift by using Vertex AI Monitoring.  

**Video Walkthrough of this notebook:**

Includes conversational walkthrough and more explanatory information than the notebook:

<p align="center" width="100%"><center><a href="https://youtu.be/kzDd94KucBQ" target="_blank" rel="noopener noreferrer"><img src="../architectures/thumbnails/playbutton/03tools_pipe2.png" width="40%"></a></center></p>

**Prerequisites:**
-  [03a - BQML Logistic Regression](03a%20-%20BQML%20Logistic%20Regression.ipynb)
-  [03Tools - Predictions](03Tools%20-%20Predictions.ipynb)

**Resources:**
- Pipelines:
    - [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)
        - [Kubeflow](https://www.kubeflow.org/docs/components/pipelines/v2/author-a-pipeline/)
            - [Install the Kubeflow Pipelines SDK](https://www.kubeflow.org/docs/components/pipelines/v1/sdk/install-sdk/)
        - [Google Cloud Pre-Built Pipeline Components](https://cloud.google.com/vertex-ai/docs/pipelines/gcpc-list)
            - [SDK Reference](https://google-cloud-pipeline-components.readthedocs.io)
- [BigQuery](https://cloud.google.com/bigquery)
    - [Documentation:](https://cloud.google.com/bigquery/docs/query-overview)
    - [API:](https://cloud.google.com/bigquery/docs/reference/libraries-overview)
        - [Clients](https://cloud.google.com/bigquery/docs/reference/libraries)
            - [Python SDK:](https://github.com/googleapis/python-bigquery)
            - [Python Library Reference:](https://cloud.google.com/python/docs/reference/bigquery/latest)
- [Vertex AI](https://cloud.google.com/vertex-ai)
    - [Documentation:](https://cloud.google.com/vertex-ai/docs/start/introduction-unified-platform)
    - [API:](https://cloud.google.com/vertex-ai/docs/reference)
        - [Clients:](https://cloud.google.com/vertex-ai/docs/start/client-libraries)
            - [Python SDK:](https://github.com/googleapis/python-aiplatform)
            - [Python Library Reference:](https://cloud.google.com/python/docs/reference/aiplatform/latest)

**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/03tools_pipe2_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/03tools_pipe2_console.png" width="45%">
</p>

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
EXPERIMENT = 'pipeline_ex2'
SERIES = '03'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [3]:
from google.cloud import aiplatform
from google.cloud import bigquery

from datetime import datetime
from typing import NamedTuple

import kfp # used for dsl.pipeline
import kfp.v2.dsl as dsl # used for dsl.component, dsl.Output, dsl.Input, dsl.Artifact, dsl.Model, ...
from google_cloud_pipeline_components import aiplatform as gcc_aip

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()

parameters:

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}/pipelines"
RUN_NAME = f'run-{TIMESTAMP}'
DIR = f"temp/{EXPERIMENT}"

In [6]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

List the service accounts current roles:

In [7]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/bigquery.admin
roles/owner
roles/run.admin
roles/storage.objectAdmin


>Note: If the resulting list is missing [roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles) then [revisit the setup notebook](../00%20-%20Setup/00%20-%20Environment%20Setup.ipynb#permissions) and add this permission to the service account with the provided instructions.

environment:

In [8]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Custom Components (KFP)

Vertex AI Pipelines are made up of components that run independently with inputs and outputs that connect to form a graph - the pipeline.  For this notebook workflow the following custom components are used to orchestrate the training of a challenger model, evaluating the challenger and an existing model, comparing them based on model metrics, if the challenger is better then replace the model already deployed on an existing endpoint.  These custom components are constructed as python functions!

### Get The Deployed Model

In [9]:
@dsl.component(
    base_image = "python:3.9",
    packages_to_install = ['google-cloud-aiplatform']
)
def get_deployed_model(
    project: str,
    region: str,
    series: str,
    bqml_model: dsl.Output[dsl.Artifact],
    vertex_endpoint: dsl.Output[dsl.Artifact]
):

    # setup
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)

    # retrieve (or create) endpoint
    endpoints = aiplatform.Endpoint.list(filter = f"display_name={series} AND labels.series={series}")
    if endpoints:
        endpoint = endpoints[0]
        print(f"Endpoint Exists: {endpoints[0].resource_name}")
    else:
        endpoint = aiplatform.Endpoint.create(
            display_name = f"{series}",
            labels = {'series' : f"{series}"}    
        )
        print(f"Endpoint Created: {endpoint.resource_name}")
    
    # retrieve deployed model with most traffic and get BQML model name
    traffic_split = endpoint.traffic_split
    if traffic_split:
        deployed_model_id = max(traffic_split, key = traffic_split.get)
        if deployed_model_id:
            for model in endpoint.list_models():
                if model.id == deployed_model_id:
                    deployed_model = model.model+f'@{model.model_version_id}'
            deployed_model = aiplatform.Model(model_name = deployed_model)
            bq_model = deployed_model.display_name+f"_{deployed_model.labels['timestamp']}"
        else: bq_model = 'none'
    else: bq_model = 'none'
    
    bqml_model.uri = bq_model 
    vertex_endpoint.uri = endpoint.resource_name

### Model Metrics
- Get Predictions for Test data from BigQuery Model
- Calculate [average precision for the precision-recall curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score)

In [10]:
@dsl.component(
    base_image = 'python:3.9',
    packages_to_install = ['pandas','db-dtypes','pyarrow','sklearn','google-cloud-bigquery']
)
def bqml_eval(
    project: str,
    region: str,
    var_target: str,
    bq_project: str,
    bq_dataset: str,
    bq_table: str,
    bqml_model: dsl.Input[dsl.Model],
    metrics: dsl.Output[dsl.Metrics],
    metricsc: dsl.Output[dsl.ClassificationMetrics]
) -> NamedTuple("model_eval", [("metric", float)]):

    # setup
    from collections import namedtuple
    from sklearn.metrics import average_precision_score, confusion_matrix
    from google.cloud import bigquery
    bq = bigquery.Client(project = project)

    if bqml_model.uri != 'none':
        query = f"""
        SELECT {var_target}, predicted_{var_target}, prob, splits 
        FROM ML.PREDICT (MODEL `{bq_project}.{bq_dataset}.{bqml_model.uri}`,(
            SELECT *
            FROM `{bq_project}.{bq_dataset}.{bq_table}`
            WHERE splits = 'TEST')
          ), UNNEST(predicted_{var_target}_probs)
        WHERE label=1
        """
        pred = bq.query(query = query).to_dataframe()

        auPRC = average_precision_score(pred[var_target].astype(int), pred['prob'], average='micro')    
        metrics.log_metric('auPRC', auPRC)
        metricsc.log_confusion_matrix(['Not Fraud', 'Fraud'], confusion_matrix(pred[var_target].astype(int), pred[f'predicted_{var_target}'].astype(int)).tolist())
    else:
        auPRC = 0.0
        metrics.log_metric('auPRC', auPRC)
    
    model_eval = namedtuple("model_eval", ["metric"])
    return model_eval(metric = float(auPRC))

### BigQuery - Train DNN

In [11]:
@dsl.component(
    base_image = 'python:3.9',
    packages_to_install = ['google-cloud-bigquery']
)
def bqml_dnn(
    project: str,
    region: str,
    series: str,
    experiment: str,
    timestamp: str,
    var_target: str,
    var_omit: str,
    bq_project: str,
    bq_dataset: str,
    bq_table: str,
    bqml_model: dsl.Output[dsl.Artifact]
) -> NamedTuple("bqml_training", [("query", str)]):
    
    from collections import namedtuple
    from google.cloud import bigquery
    bq = bigquery.Client(project = project)
    
    bq_model = f'{series}_{experiment}_{timestamp}'
    query = f"""
    CREATE OR REPLACE MODEL `{bq_project}.{bq_dataset}.{bq_model}`
    OPTIONS
        (model_type = 'DNN_CLASSIFIER',
            auto_class_weights = FALSE,
            input_label_cols = ['{var_target}'],
            data_split_col = 'custom_splits',
            data_split_method = 'CUSTOM',
            EARLY_STOP = FALSE,
            OPTIMIZER = 'SGD',
            HIDDEN_UNITS = [64, 32],
            LEARN_RATE = 0.001,
            ACTIVATION_FN = 'SIGMOID',
            MAX_ITERATIONS = 15,
            HPARAM_TUNING_ALGORITHM = 'VIZIER_DEFAULT',
            HPARAM_TUNING_OBJECTIVES = ['ROC_AUC'],
            DROPOUT = HPARAM_RANGE(0, 0.8),
            BATCH_SIZE = HPARAM_RANGE(8, 500),
            MAX_PARALLEL_TRIALS = 5,
            NUM_TRIALS = 10
        ) AS
    SELECT * EXCEPT({','.join(var_omit.split())}, splits),
        CASE
            WHEN splits = 'VALIDATE' THEN 'EVAL'
            ELSE splits
        END AS custom_splits
    FROM `{bq_project}.{bq_dataset}.{bq_table}`
    WHERE splits != 'TEST'
    """
    job = bq.query(query = query)
    job.result()
    bqml_model.uri = bq_model
    
    result = namedtuple("bqml_training", ["query"])
                
    return result(query = str(query))

### Compare Models

In [12]:
@dsl.component
def model_compare(
    base_metric: float,
    challenger_metric: float,
) -> bool: 
    
    if base_metric < challenger_metric:
        replace = True
    else:
        replace = False
    
    return replace

### Export BQML Model

In [13]:
@dsl.component(
    base_image = 'python:3.9',
    packages_to_install = ['google-cloud-bigquery', 'google-cloud-aiplatform']
)
def bqml_export(
    project: str,
    region: str,
    series: str,
    experiment: str,
    timestamp: str,
    uri: str,
    run_name: str,
    bq_project: str,
    bq_dataset: str,
    bqml_model: dsl.Input[dsl.Model],
    tf_model: dsl.Output[dsl.Artifact],
    vertex_model: dsl.Output[dsl.Artifact]
):
    
    # hardcoded parameter
    deploy_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest'
    
    # setup
    from google.cloud import bigquery
    bq = bigquery.Client(project = project)
    
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)
    
    # export BQML Challenger Model
    query = f"""
    EXPORT MODEL `{bq_project}.{bq_dataset}.{bqml_model.uri}`
        OPTIONS (URI = '{uri}/models/{timestamp}/model')
    """
    export = bq.query(query = query)
    export.result()
    
    # upload model to Vertex AI Model Registry
    modelmatch = aiplatform.Model.list(filter = f'display_name={series}_{experiment} AND labels.series={series} AND labels.experiment={experiment}')

    upload_model = True
    if modelmatch:
        print("Model Already in Registry:")
        if run_name in modelmatch[0].version_aliases:
            print("This version already loaded, no action taken.")
            upload_model = False
            model = aiplatform.Model(model_name = modelmatch[0].resource_name)
        else:
            print('Loading model as new default version.')
            parent_model =  modelmatch[0].resource_name
    else:
        print('This is a new model, adding to model registry as version 1')
        parent_model = ''

    if upload_model:
        model = aiplatform.Model.upload(
            display_name = f'{series}_{experiment}',
            model_id = f'model_{series}_{experiment}',
            parent_model = parent_model,
            serving_container_image_uri = deploy_image,
            artifact_uri = f"{uri}/models/{timestamp}/model",
            is_default_version = True,
            version_aliases = [run_name],
            version_description = run_name,
            labels = {'series' : f'{series}', 'experiment' : f'{experiment}', 'timestamp': f'{timestamp}', 'run_name' : f'{run_name}'}
        )  
    
    tf_model.uri = f'{uri}/models/{timestamp}/model'
    vertex_model.uri = model.resource_name

### Replace Model On Endpoint

In [14]:
@dsl.component(
    base_image = 'python:3.9',
    packages_to_install = ['google-cloud-aiplatform']
)
def endpoint_update(
    project: str,
    region: str,
    series: str,
    experiment: str,
    vertex_endpoint: dsl.Input[dsl.Artifact],
    vertex_model: dsl.Input[dsl.Model]
):
    
    # hardcoded parameter
    deploy_compute = 'n1-standard-4'
    
    # setup
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)

    # retrieve endpoint
    endpoint = aiplatform.Endpoint(vertex_endpoint.uri)
    
    # retrieve model
    model = aiplatform.Model(vertex_model.uri)
    
    # deploy model to endpoint with 100% traffic
    endpoint.deploy(
        model = model,
        deployed_model_display_name = model.display_name,
        traffic_percentage = 100,
        machine_type = deploy_compute,
        min_replica_count = 1,
        max_replica_count = 1
    )
    
    # remove 0 traffic models
    for deployed_model in endpoint.list_models():
        if deployed_model.id in endpoint.traffic_split:
            print(f"Model {deployed_model.display_name} with version {deployed_model.model_version_id} has traffic = {endpoint.traffic_split[deployed_model.id]}")
        else:
            endpoint.undeploy(deployed_model_id = deployed_model.id)
            print(f"Undeploying {deployed_model.display_name} with version {deployed_model.model_version_id} because it has no traffic.")

---
## Pipeline (KFP) Definition

In [15]:
@dsl.pipeline(
    name = f'series-{SERIES}-endpoint-challenger',
    description = 'Update endpoint with challenger model (conditionally).'
)
def pipeline(
    project: str,
    region: str,
    series: str,
    experiment: str,
    timestamp: str,
    bq_project: str,
    bq_dataset: str,
    bq_table: str,
    var_target: str,
    var_omit: str,
    uri: str,
    run_name: str
):
   
    # get the current model
    current_model = get_deployed_model(
        project = project,
        region = region,
        series = series
    ).set_display_name('Get Current Model').set_caching_options(False)

    # get AUC for current model
    base_model_eval = bqml_eval(
        project = project,
        region = region,
        var_target = var_target,
        bq_project = bq_project,
        bq_dataset = bq_dataset,
        bq_table = bq_table,
        bqml_model = current_model.outputs['bqml_model']
    ).set_display_name('Metric for Current Model').set_caching_options(False)
    
    # train challenger model with BQML
    challenger_model = bqml_dnn(
        project = project,
        region = region,
        series = series,
        experiment = experiment,
        timestamp = timestamp,
        var_target = var_target,
        var_omit = var_omit,
        bq_project = bq_project,
        bq_dataset = bq_dataset,
        bq_table = bq_table
    ).set_display_name('Train Challenger Model').set_caching_options(True)
    
    # get AUC for challenger model
    challenger_model_eval = bqml_eval(
        project = project,
        region = region,
        var_target = var_target,
        bq_project = bq_project,
        bq_dataset = bq_dataset,
        bq_table = bq_table,
        bqml_model = challenger_model.outputs['bqml_model']
    ).set_display_name('Metric for Challenger Model').set_caching_options(False)
    challenger_model_eval.after(challenger_model)
    
    # compare models
    compare = model_compare(
        base_metric = base_model_eval.outputs["metric"],
        challenger_metric = challenger_model_eval.outputs["metric"]
    ).set_display_name('Compare Models')
    
    # conditional deployment
    with dsl.Condition(
        compare.output == 'true',
        name = "replace_model"
    ):
        # export BQML model to Vertex AI Model Registry
        export = bqml_export(
            project = project,
            region = region,
            series = series,
            experiment = experiment,
            timestamp = timestamp,
            uri = uri,
            run_name = run_name,
            bq_project = bq_project,
            bq_dataset = bq_dataset,
            bqml_model = challenger_model.outputs["bqml_model"]
        ).set_display_name('Export BQML Model')
        
        # replace model on endpoint (03b)
        replace = endpoint_update(
            project = project,
            region = region,
            series = series,
            experiment = experiment,
            vertex_endpoint = current_model.outputs['vertex_endpoint'],
            vertex_model = export.outputs['vertex_model']
        ).set_display_name('Deploy The Challenger Model')

---
## Compile And Run Pipeline

### Compile Inputs

In [16]:
parameter_values = {
    "project" : PROJECT_ID,
    "region" : REGION,
    "series": SERIES,
    "experiment" : EXPERIMENT,
    "timestamp": TIMESTAMP,
    "bq_project": BQ_PROJECT,
    "bq_dataset": BQ_DATASET,
    "bq_table": BQ_TABLE,
    "var_target": VAR_TARGET,
    "var_omit": VAR_OMIT,
    "uri": URI,
    "run_name": RUN_NAME
}

### Compile Pipeline

In [17]:
# from kfp.v2 import compiler
kfp.v2.compiler.Compiler().compile(
    pipeline_func = pipeline,
    package_path = f"{DIR}/{EXPERIMENT}.json"
)

/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


### Define Pipeline Job

Using compiled pipeline:

In [18]:
pipeline_job = aiplatform.PipelineJob(
    display_name = f'{EXPERIMENT}',
    template_path = f"{DIR}/{EXPERIMENT}.json",
    pipeline_root = f"{URI}/pipeline_root",
    parameter_values = parameter_values,
    enable_caching = False, # overrides all component/task settings
    labels = {'series': SERIES, 'experiment': EXPERIMENT}
)

### Submit Pipeline Job

In [19]:
response = pipeline_job.submit(
    service_account = SERVICE_ACCOUNT
)

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/series-03-endpoint-challenger-20221020144506
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/series-03-endpoint-challenger-20221020144506')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/series-03-endpoint-challenger-20221020144506?project=1026793852137


Using the following link to view the job in the GCP console:

In [21]:
print(f'The Dashboard can be viewed here:\n{pipeline_job._dashboard_uri()}')

The Dashboard can be viewed here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/series-03-endpoint-challenger-20221020144506?project=1026793852137


#### Wait On Pipeline Job

In [22]:
pipeline_job.wait()

PipelineJob run completed. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/series-03-endpoint-challenger-20221020144506


### Retrieve Pipeline Information

In [23]:
aiplatform.get_pipeline_df(pipeline = f'series-{SERIES}-endpoint-challenger')

,pipeline_name,run_name,param.input:bq_project,param.input:bq_table,param.input:series,param.input:experiment,param.input:var_omit,param.input:region,param.input:var_target,param.input:run_name,param.input:project,param.input:bq_dataset,param.input:uri,param.input:timestamp,metric.auPRC,metric.confusionMatrix
0,series-03-endpoint-challenger,series-03-endpoint-challenger-20221020144506,statmike-mlops-349915,fraud_prepped,03,pipeline_ex2,transaction_id,us-central1,Class,run-20221020144504,statmike-mlops-349915,fraud,gs://statmike-mlops-349915/03/pipeline_ex2/pip...,20221020144504,0.843390,"{'rows': [{'row': [28451.0, 4.0]}, {'row': [12..."
1,series-03-endpoint-challenger,series-03-endpoint-challenger-20221020121726,statmike-mlops-349915,fraud_prepped,03,pipeline_ex2,transaction_id,us-central1,Class,run-20221020121715,statmike-mlops-349915,fraud,gs://statmike-mlops-349915/03/pipeline_ex2/pip...,20221020121715,0.828472,"{'rows': [{'row': [28453.0, 2.0]}, {'row': [22..."
2,series-03-endpoint-challenger,series-03-endpoint-challenger-20221004192647,statmike-mlops-349915,fraud_prepped,03,pipeline_ex2,transaction_id,us-central1,Class,run-20221004192552,statmike-mlops-349915,fraud,gs://statmike-mlops-349915/03/pipeline_ex2/pip...,20221004192552,0.818403,{'annotationSpecs': [{'displayName': 'Not Frau...
3,series-03-endpoint-challenger,series-03-endpoint-challenger-20221004152232,statmike-mlops-349915,fraud_prepped,03,pipeline_ex2,transaction_id,us-central1,Class,run-20221004152128,statmike-mlops-349915,fraud,gs://statmike-mlops-349915/03/pipeline_ex2/pip...,20221004152128,0.821983,{'annotationSpecs': [{'displayName': 'Not Frau...


## Review Pipeline Run

<p aligh="center"><center><img src="../architectures/notebooks/03/pipeline_ex2.png" width="75%"></center></p>

---
## Remove Resources
see notebook "99 - Cleanup"